# Part 1: Mathematical Programming

In [7]:
from distutils.command.build_scripts import first_line_re
from tkinter.tix import COLUMN
import pandas as pd
import numpy as np
# from webob import second

smallFilePath = '../cloud_resource_allocation/small.csv'
largeFilePath = '../cloud_resource_allocation/large.csv'

class CloudResourceAllocation:
    '''For each instance, we have following fields: 
            the 1st line of the csv files contains the number of jobs N,
            2nd line contains the CPU capacity Q1, and memory capacity Q2, 
            3rd line onwards contains the 
                    ID, CPU demand, memory demand,
                    and payment of a job.
    '''
    def __init__(self, filePath):
        with open(filePath, 'r') as file:
            first_line = file.readline()
            second_line = file.readline()
            self.N = int(first_line.split(',')[0])
            self.Q1,self.Q2 = int(second_line.split(',')[0]),int(second_line.split(',')[1])
            
        self.features = pd.read_csv(filePath, skiprows=range(2), header=None)
        self.features.columns = ['ID', 'CPUDemand', 'MemoryDemand','payment']
    
    def get_features(self):
        return self.features_left
    
    def __str__(self) -> str:
        return f'N: {self.N}, CPU Capacity Q1: {self.Q1}, Memort Capacity Q2: {self.Q2}, \n features_left:\n{self.features}'

In [8]:
small = CloudResourceAllocation(smallFilePath)
large = CloudResourceAllocation(largeFilePath)

# large = CloudResourceAllocation(largeFilePath)
print(small.__str__())
print('-'*70)
print(large.__str__())

N: 10, CPU Capacity Q1: 1000, Memort Capacity Q2: 2000, 
 features_left:
   ID  CPUDemand  MemoryDemand  payment
0   1        158           317      836
1   2        136           280      607
2   3        114            61      724
3   4        203           372      269
4   5          8            11      381
5   6        183           189       24
6   7         66            66      675
7   8         41           391      907
8   9        144           377      962
9  10         36           297      531
----------------------------------------------------------------------
N: 1000, CPU Capacity Q1: 10000, Memort Capacity Q2: 10000, 
 features_left:
       ID  CPUDemand  MemoryDemand  payment
0       1       1573          1581      836
1       2       1359          1397      607
2       3       1131           304      724
3       4       2025          1857      269
4       5         71            51      381
..    ...        ...           ...      ...
995   996        592           

# Part 1 
The goal of the resource allocation problem is to decide which job to accept (and which to decline), so that the CPU and memory capacity is not exceeded by the accepted jobs, and the total charged payment is maximised

In [9]:
# branch and bound 
def bounding(values, weights, capacity):
    bound = 0
    remaining_capacity = capacity
    
    efficiency = [values[i] / weights[i] for i in range(len(values))]
    
    sorted_idx = sorted(range(len(efficiency)), reverse=True, key=efficiency.__getitem__)
    
    for i in sorted_idx:
        if weights[i] > remaining_capacity:
            fraction = remaining_capacity / weights[i]
            frac_value = fraction * values[i]
            bound += frac_value
            return bound
        else:
            bound += values[i]
            remaining_capacity -= weights[i]
    
    return bound

In [ ]:
# Import deque for the stack structure, copy for deep copy nodes
from collections import deque
import copy

def knapsack_bb_dfs(values, weights, capacity):
    # Initialise the root, where 'expanded_item' indicates the item to be expanded at this node
    root = {
        'solution': [0] * len(values),
        'value': 0,
        'weight': 0,
        'expanded_item': 0
    }
    
    # Initially, the fringe contains the root node only
    best_solution = root
    fringe = deque()
    fringe.append(root)
    
    while len(fringe) > 0:
        # Depth-first-search, Last-In-First-Out of the stack
        node = fringe.pop()
        
        # Check if the node is a leaf node
        if node['expanded_item'] == len(values):
            if node['value'] > best_solution['value']:
                best_solution = node
                continue
        
        # Obtain the sub-problem: values, weights, capacity
        node_values = values[node['expanded_item']:]
        node_weights = weights[node['expanded_item']:]
        node_capacity = capacity - node['weight']
        
        # Bounding on the sub-problem, and then add the value of the current solution
        bound = bounding(node_values, node_weights, node_capacity) + node['value']
        
        # Prune the branch
        if bound <= best_solution['value']:
            continue
            
        # Branching on the expanded item, 0 or 1
        expanded_item = node['expanded_item']
        
        # Child 1: unselect the expanded item
        child1 = copy.deepcopy(node)
        child1['solution'][expanded_item] = 0
        child1['expanded_item'] = expanded_item + 1
        fringe.append(child1)
        
        # Child 2: select the expanded item if the capacity is enough
        new_weights = node['weight'] + weights[expanded_item]
        
        if new_weights <= capacity:
            child2 = copy.deepcopy(node)
            child2['solution'][expanded_item] = 1
            child2['value'] = node['value'] + values[expanded_item]
            child2['weight'] = new_weights
            child2['expanded_item'] = expanded_item + 1
            fringe.append(child2)
    
    return best_solution
